# Cross validation for BigConTest

In [1]:
import numpy as np
import pandas as pd

## Load Data

In [2]:
train = pd.read_pickle("../all_df.p")
train.head()

,날짜,순위,영화명,개봉일,매출액,매출액 점유율,매출액증감 (전일대비),매출액증감율 (전일대비),누적매출액,관객수,...,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
4,2014년 08월 07일(목),1,명량,2014-07-30 00:00:00,4892511700,0.595,-380933700,-0.1,61033902816,652776,...,1239,6776,한국,한국,(주)빅스톤픽쳐스,씨제이이앤엠(주),15세이상관람가,"사극,액션",김한민,"최민식,류승룡,조진웅,김명곤,진구,이정현,권율,노민우,김태훈,이승준,오타니 료헤이,..."
5,2014년 08월 07일(목),2,해적: 바다로 간 산적,2014-08-06 00:00:00,1890503200,0.23,-130889400,-0.1,4137374300,254862,...,795,3932,한국,한국,(주)하리마오픽쳐스,롯데쇼핑㈜롯데엔터테인먼트,12세이상관람가,어드벤처,이석훈,"김남길,손예진,유해진,이경영,오달수,김태우,박철민,신정근,김원해,조달환,조희봉,정성..."
6,2014년 08월 07일(목),3,드래곤 길들이기 2,2014-07-23 00:00:00,437352300,0.053,-27892900,-0.1,19775433889,58182,...,423,1016,미국,미국,드림웍스 애니메이션,씨제이이앤엠(주),전체관람가,"애니메이션,액션,어드벤처",딘 데블로이스,"제이 바루첼,제라드 버틀러,케이트 블란쳇,아메리카 페레라"
7,2014년 08월 07일(목),4,가디언즈 오브 갤럭시,2014-07-31 00:00:00,347508400,0.042,-11947100,0,7187518884,39837,...,344,844,미국,미국,마블 스튜디오,소니픽쳐스릴리징월트디즈니스튜디오스코리아(주),12세이상관람가,"액션,어드벤처,SF",제임스 건,"크리스 프랫,조 샐다나,데이브 바티스타,빈 디젤,브래들리 쿠퍼"
8,2014년 08월 07일(목),5,명탐정 코난 : 이차원의 저격수,2014-08-06 00:00:00,262921800,0.032,103795200,0.7,433027400,38912,...,261,514,일본,일본,TMS 엔터테인먼트,씨제이이앤엠(주),12세이상관람가,"애니메이션,스릴러,어드벤처",시즈노 코분,"타카야마 미나미, 히다카 노리코"


## Preprocessing

In [3]:
from datetime import datetime
import re

In [4]:
train = train.drop(['매출액','매출액 점유율','매출액증감 (전일대비)','매출액증감율 (전일대비)','관객수','관객수증감 (전일대비)',
                    '관객수증감율 (전일대비)','대표국적','국적','제작사','배급사','등급','장르','감독','배우'], axis=1)
train = train.dropna()
train.head()

,날짜,순위,영화명,개봉일,누적매출액,누적관객수,스크린수,상영횟수
4,2014년 08월 07일(목),1,명량,2014-07-30 00:00:00,61033902816,7982455,1239,6776
5,2014년 08월 07일(목),2,해적: 바다로 간 산적,2014-08-06 00:00:00,4137374300,563642,795,3932
6,2014년 08월 07일(목),3,드래곤 길들이기 2,2014-07-23 00:00:00,19775433889,2460400,423,1016
7,2014년 08월 07일(목),4,가디언즈 오브 갤럭시,2014-07-31 00:00:00,7187518884,856783,344,844
8,2014년 08월 07일(목),5,명탐정 코난 : 이차원의 저격수,2014-08-06 00:00:00,433027400,63785,261,514


In [5]:
train["날짜"] = train["날짜"].apply(lambda date: re.sub('[(년월화수목금토일)]', '', date))
train["날짜"] = train["날짜"].apply(lambda date: date.replace(" ", "-"))
train["날짜"].head(3)

4    2014-08-07
5    2014-08-07
6    2014-08-07
Name: 날짜, dtype: object

In [6]:
train["개봉일"] = pd.to_datetime(train["개봉일"], errors='coerce')
train["개봉일"].head()

4   2014-07-30
5   2014-08-06
6   2014-07-23
7   2014-07-31
8   2014-08-06
Name: 개봉일, dtype: datetime64[ns]

In [7]:
train["상영기간"] = pd.to_datetime(train["날짜"]) - pd.to_datetime(train["개봉일"])
train["상영기간"]

4        8 days
5        1 days
6       15 days
7        7 days
8        1 days
9       15 days
10       1 days
11      14 days
12       7 days
13       0 days
14      -6 days
15       0 days
16      -6 days
17       0 days
18       7 days
19       7 days
20       0 days
21     -13 days
22       1 days
23       1 days
24      21 days
25       1 days
27      28 days
28     -13 days
29      77 days
31       8 days
33       0 days
36      -7 days
37      56 days
38     504 days
         ...   
727     22 days
728      1 days
729   7909 days
730      1 days
732     78 days
733     58 days
734    128 days
735      1 days
737      8 days
738   5405 days
739     22 days
740   7951 days
741     51 days
744     36 days
745      8 days
746     99 days
747     15 days
749     29 days
750     36 days
751     29 days
752    695 days
753    366 days
754     23 days
755     43 days
756   7323 days
757     50 days
758     29 days
759     64 days
760      1 days
761     43 days
Name: 상영기간, dtype: timed

In [8]:
train = train[train["상영기간"] >= '0 days']
train = train[train["상영기간"] < '5000 days']
train["상영기간"] = train["상영기간"].astype('timedelta64[D]').astype(int)
train.head()

,날짜,순위,영화명,개봉일,누적매출액,누적관객수,스크린수,상영횟수,상영기간
4,2014-08-07,1,명량,2014-07-30,61033902816,7982455,1239,6776,8
5,2014-08-07,2,해적: 바다로 간 산적,2014-08-06,4137374300,563642,795,3932,1
6,2014-08-07,3,드래곤 길들이기 2,2014-07-23,19775433889,2460400,423,1016,15
7,2014-08-07,4,가디언즈 오브 갤럭시,2014-07-31,7187518884,856783,344,844,7
8,2014-08-07,5,명탐정 코난 : 이차원의 저격수,2014-08-06,433027400,63785,261,514,1


In [17]:
feature_names = ["누적매출액", "상영횟수", "상영기간"]
label_name = "누적관객수"

train_X = train[feature_names]
train_y = train[label_name]

## Cross Validation Set

In [18]:
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold

In [19]:
def RMSE(y, ypred):
    rmse = np.sqrt(np.mean((y - ypred) ** 2))
    return rmse

In [20]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.3)
y_test.head()

648    67196
356       68
501      181
416      245
220     2436
Name: 누적관객수, dtype: object

## Predict

In [21]:
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor

In [22]:
model_svr = LinearSVR(C=1.0, max_iter=100)
model_rfr = RandomForestRegressor(n_estimators=100, n_jobs=-1)
model_rfr.fit(X_train, y_train)
model_svr.fit(X_train, y_train)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=100,
     random_state=None, tol=0.0001, verbose=0)

In [23]:
y_pred_rfr = model_rfr.predict(X_test)
y_pred_svr = model_svr.predict(X_test)
y_pred_rfr

array([  6.73978200e+04,   7.32600000e+01,   2.08530000e+02, ...,
         2.82993818e+06,   4.55969730e+05,   6.87455000e+03])

In [24]:
error_rfr = RMSE(y_test, y_pred_rfr)
error_svr = RMSE(y_test, y_pred_svr)
print("RandomForest RMSE = {0:.6f}".format(error_rfr))
print("LinearSVR RMSE = {0:.6f}".format(error_svr))

RandomForest RMSE = 31817.049237
LinearSVR RMSE = 161836.026577


## Result
- RandomForest RMSE = 31817.049237
- LinearSVR RMSE = 161836.026577